In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install --quiet unstructured

In [ ]:
!pip install --quiet chromadb

In [ ]:
!apt-get install poppler-utils -y

In [ ]:
!pip install --quiet --upgrade unstructured[local-inference]

In [ ]:
#from unstructured_client import UnstructuredClient
#from unstructured_client.models import shared
#from unstructured_client.models.errors import SDKError

from unstructured.chunking.title import chunk_by_title
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
#from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements
#from pdf2image import convert_from_path

#from pdf2image.exceptions import (
#PDFInfoNotInstalledError,
#PDFPageCountError,
#PDFSyntaxError
#)

import chromadb

In [ ]:
cons_GenAI = partition_html(filename="/kaggle/input/example-files/Generative AI_ The Next Consumer Platform _ Andreessen Horowitz.html")
econ_GenAI = partition_html(filename="/kaggle/input/example-files/The Economic Case for Generative AI.html")
pros_GenAI = partition_html(filename="/kaggle/input/example-files/The Future of Prosumer_ The Rise of AI Workflows .html")

In [ ]:
filename = "/kaggle/input/pdf-example/attention-is-all-you-need.pdf"

pdf_elements = partition_pdf(
    filename = "/kaggle/input/pdf-example/attention-is-all-you-need.pdf",
    strategy = "hi_res",
    pdf_infer_table_structure=True
)

In [ ]:
#Remove the footers and references
footers = [item for item in pdf_elements if item.category=="Footer"]
reference = [item for item in pdf_elements if item.category=="Title" and item.text=="References"]
reference_id = reference[0].to_dict()['element_id']
reference_pg = reference[0].to_dict()['metadata']['page_number']
reference_items = [item for item in pdf_elements if item.metadata.parent_id == reference_id or item.metadata.page_number>reference_pg]
pdf_adj_elements = [item for item in pdf_elements if item not in footers+reference+reference_items]

In [ ]:
el_set = set()
for el in pdf_adj_elements:
    el_set.add(el.category)
print(el_set)

In [ ]:
#Chunk by title and add to ChromaDB
elements = chunk_by_title(pdf_adj_elements+cons_GenAI+econ_GenAI+pros_GenAI)

In [1]:
! pip install --quiet langchain_cohere langchain_experimental
#! pip install --upgrade --quiet  transformers huggingface_hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.34.103 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
j

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

documents=[]
for el in elements:
    metadata = el.metadata.to_dict()
    filtered_metadata = {}
    del metadata['languages']
    for key,value in metadata.items():
        if isinstance(value,list):
            continue
        else:
            filtered_metadata[key] = value
    filtered_metadata['source'] = filtered_metadata['filename']
    documents.append(Document(page_content = el.text,metadata = filtered_metadata))

In [2]:
# LLM
import os
os.environ['COHERE_API_KEY'] = '4wxxCfBV3K2rj33RYQVOZMZ9DtWVvgmI57noCUFF'

In [ ]:
from langchain_cohere import CohereEmbeddings
embd = CohereEmbeddings()
vectorstore = Chroma.from_documents(documents=documents,embedding=embd)
vector_retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs = {"k":5}
)

In [ ]:
#Set up a tool to access the vector data store
from langchain.tools.retriever import create_retriever_tool
vectorstore_search = create_retriever_tool(
    retriever=vector_retriever,
    name="GenAI_content_expert",
    description="Retrieve relevant information from a vectorstore that contains expert information on GenAI and its industry usage"
)

In [ ]:
!pip install --quiet duckduckgo-search

In [ ]:
#Set up a tool to access internet search results
from langchain_community.tools import DuckDuckGoSearchResults
internet_search = DuckDuckGoSearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."

from langchain_core.pydantic_v1 import BaseModel,Field
class DDGoSearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = DDGoSearchInput

In [ ]:
#Set up a tool to execute a python script
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput

In [3]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

In [4]:
#Create an agent using the Cohere ReAct framework with access to the relevant tools for executing planned multi-tool use design pattern
llm = ChatCohere(model='command-r-plus',temperature=0.3)

#Preamble
preamble = """
You are a learning companion who answers any questions that the user has on the field of Generative AI and generates the relevant results.
You are equipped with a vectorstore to search for the relevant information in crafting an understandable response.
You are also equipped with an internet search tool to look up the information if the vectorstore provides limited context.
If some numerical calculations or visualizations are required, you are equipped with the python interpreter for generating the results.
You also have a text to speech tool to generate audio output.
If the user asks for something not related to generative AI, please politely draw attention towards the objective of learning generative AI
"""

prompt = ChatPromptTemplate.from_template('{input}')

In [15]:
from langchain.chains import LLMChain
from langchain_core.tools import tool

@tool
def expand_query(query):
    """Augments the given query by generating multiple related questions that would more reliably cover the vector embedding search space"""
    expand_prompt = ChatPromptTemplate.from_template("""You are an assistant that helps generate the relevant educational content based on user requirements.
        For the query enclosed in triple backticks, please suggest five additional questions related to the context of the information requested.
        Suggest only short questions without compound sentences. Make sure that the questions touch upon different aspects of the query.
        Output one question per line and append a ? symbol at the end of every question. Do not number the questions
        ```{query}```""")
    augment_chain = LLMChain(llm=llm,prompt=expand_prompt)
    response = augment_chain.run(query)
    return response.split('\n\n')
    
expand_query.name = "augment_query_with_expansion"
expand_query.description = "Augments the given query by generating multiple related questions that would more reliably cover the vector embedding search space"

from langchain_core.pydantic_v1 import BaseModel, Field
class primary_query(BaseModel):
    query: str = Field(description="The original query corresponding to user input")
expand_query.args_schema = primary_query

In [ ]:
agent = create_cohere_react_agent(
    llm = llm,
    prompt=prompt,
    tools=[vectorstore_search,internet_search,python_tool,tts_co_tool]
)

agent_executor = AgentExecutor(agent=agent,tools=[vectorstore_search,internet_search,python_tool,tts_co_tool],verbose=True)

In [ ]:
response = agent_executor.invoke({
    "input":"What was the very first LLM model designed and how long ago was it designed?",
    "preamble":preamble
})

response

In [ ]:
response['output']

In [ ]:
from transformers import load_tool
tts_tool = load_tool("text-to-speech")
tts_tool(response['output'])